## What the App Does (High-Level Overview)

This program is a desktop translation application that allows users to:
- Translate text between languages (auto-detect or manually select).
- Speak into a microphone → convert speech to text.
- Speak → instantly hear the translated version (speech-to-speech).
- Perform real-time continuous translation.
- Listen to translated text using text-to-speech (TTS).
- Store translation history in a CSV file.
The app uses:
- Tkinter → graphical interface
- GoogleTranslator (from deep_translator) → translation engine
- SpeechRecognition → captures and converts audio input to text
- gTTS → converts translated text back to speech
- playsound → plays the generated audio
- langdetect → automatically detects the language of the text input
- CSV → stores translation history

## Imports and Setup

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
from deep_translator import GoogleTranslator
from gtts import gTTS
from langdetect import detect
import speech_recognition as sr
import pyperclip
import threading
import csv
import time
import os
from playsound import playsound

## Explanation:

- tkinter — The GUI framework.
- ttk — Provides themed widgets (modern look).
- messagebox — Displays pop-up dialogs (warnings, errors, etc.).
- scrolledtext — A text box with built-in scrollbar.
- deep_translator.GoogleTranslator — A wrapper to use Google Translate API.
- gTTS — Converts text into speech (audio file).
- langdetect.detect — Automatically determines the language of input text.
- speech_recognition — Converts speech (from microphone) into text.
- pyperclip — Copies text to the system clipboard.
- threading — Runs long operations (like listening or audio playback) in background threads so the GUI doesn’t freeze.
- csv, os, time — Handle file operations and logging.
- playsound — Plays audio files.

## Configuration & Language Mapping

In [ ]:
HISTORY_FILE = "translation_history.csv"

LANGUAGES = {
    'Auto Detect': 'auto',
    'English': 'en', 'French': 'fr', 'Spanish': 'es', 'German': 'de',
    'Italian': 'it', 'Portuguese': 'pt', 'Arabic': 'ar',
    'Chinese (Simplified)': 'zh-CN', 'Japanese': 'ja',
    'Korean': 'ko', 'Russian': 'ru'
}

- HISTORY_FILE: path to store all translation records.
- LANGUAGES: dictionary mapping user-friendly names → language codes used by Google Translator.

## Utility Functions
### (a) Save Translation History

In [ ]:
def save_history(mode, source_lang, target_lang, source_text, translated_text):

Writes a new row to the CSV file containing:
- timestamp
- translation mode (text, speech, real-time)
- source/target languages
- input and output text
If the file doesn’t exist, it creates one and adds column headers.

### (b) Language Detection

In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"


Uses the langdetect library to guess the language of the input text.
If detection fails (e.g., gibberish), it returns "unknown".

### (c) Play Audio

In [ ]:
def play_audio(file_path):
    threading.Thread(target=lambda: playsound(file_path)).start()

Plays sound asynchronously in a new thread so the app remains responsive.

## Translation & Speech Functions
### (a) translate_text()

Handles manual or auto translation of typed text.
Steps:
- Get text from the input box.
- Determine source and target languages.
- If set to “Auto Detect”, detect the source language using langdetect.
- Use GoogleTranslator to translate the text.
- Display translation in the output box.
- Save translation to CSV history.

### (b) copy_translation()

Copies the translated text from the output box to clipboard.

### (c) text_to_speech()

Converts translated text into an MP3 and plays it aloud:
- Reads the translated text.
- Uses gTTS with the target language.
- Saves to "translation.mp3".
- Plays it using play_audio().

### (d) speech_to_text()

Captures voice input and transcribes it to text:
- Uses SpeechRecognition’s Microphone class to record.
- Calls Google’s speech recognition API to convert to text.
- Inserts result into the input text box.

### (e) speech_to_speech()

Performs full speech → translation → speech:
- Listens for audio via microphone.
- Converts it to text.
- Detects input language.
- Translates to the selected target language.
- Converts translated text to speech.
- Plays the result.
- Saves to CSV history.

### (f) real_time_translation()

Runs continuous real-time translation until stopped (via Ctrl+C or closing):
- Constantly listens for speech in short bursts.
- Converts recognized text to translation repeatedly.
- Appends results in the output box like a live chat.
- Keeps logging each translation event.

## GUI Layout and Widgets

In [ ]:
app = tk.Tk()
app.title("AI Translator Desktop App")
app.geometry("700x650")
app.configure(bg="#f5f6fa")

Creates the main window with a custom background color and size.

### Title Section

In [ ]:
tk.Label(app, text="AI Translator", font=("Segoe UI", 18, "bold"), bg="#f5f6fa").pack(pady=10)

Simple title label at the top.

### Language Selector Section

Two dropdowns (comboboxes) for selecting source and target languages.
The default source is “Auto Detect”, and target is “French”.

### Input Text Box

A scrollable text area where users can type or receive recognized speech.

### Button Controls

Organized into two frames:

#### Text Controls:
- Translate Text
- Copy Translation
- Text-to-Speech

#### Speech Controls:
- Speech-to-Text
- Speech-to-Speech
- Real-Time Speech Translation
Each button calls its corresponding function when clicked.

### Output Area

Shows the translated text.

## Why It’s Designed This Way

| Design Choice                        | Reason                                                           |
| ------------------------------------ | ---------------------------------------------------------------- |
| **Tkinter GUI**                      | Lightweight, built-in with Python.                               |
| **deep_translator.GoogleTranslator** | No need for API key, quick setup.                                |
| **Auto Detection via `langdetect`**  | Improves user experience — no manual language selection needed.  |
| **Threading**                        | Keeps UI responsive during long operations.                      |
| **CSV Logging**                      | Simple way to keep track of translations for reuse or debugging. |
| **SpeechRecognition + gTTS**         | Provides end-to-end voice translation experience.                |
